In [1]:
import ee
import os
import sys
import random
from datetime import date
import copy
import math
import json
import pandas as pd

In [2]:
ee.Authenticate()

try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
sys.setrecursionlimit(1000000000)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=QzxyzPCgDnaUwo_qHP8OINpSqYK7lsXulVNC77PeZgk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4Ir9yLKLRyoyjju3RKtxkeWvoxnFsxAVv0-UENnD2mVRJJ0k7d5C4

Successfully saved authorization token.
The Earth Engine package initialized successfully!


In [15]:
features = ["B11", "B12", "B2", "B3", "B4", "B8", "evi", "gndvi", "ifv", "ndvi", "osavi", "ratio", "rvi"]
#features = ["B11", "B12", "B2", "B3", "B4", "B8", "evi", "gndvi", "ifv", "ndvi", "osavi", "ratio", "rvi", "ndwi", "feo", 'lai', 'soil', 'ndfia']
#features = ['pri', 'hallcover', 'cai', 'gv', 'gvs', 'npv', 'shade', 'ndfi', 'sefi', 'wefi', 'fns', 'awei',"iia", 'npv']

In [4]:
tiles = {
    'lsTilesCaat': [
      '23KNB','23KPB','23LMC','23LMG','23LMH','23LMJ','23LNC','23LND',
      '23LNE','23LNF','23LNG','23LNH','23LNJ','23LNK','23LNL','23LPC',
      '23LPD','23LPE','23LPF','23LPG','23LPH','23LPJ','23LPK','23LPL',
      '23LQC','23LQD','23LQE','23LQF','23LQG','23LQH','23LQJ','23LQK',
      '23LQL','23LRD','23LRE','23LRF','23LRG','23LRH','23LRJ','23LRK',
      '23LRL','23MNM','23MPM','23MPN','23MQM','23MQN','23MQP','23MRM',
      '23MRN','23MRP','23MRQ','23MRR','23MRS','23MRT','24LTJ','24LTK',
      '24LTL','24LTM','24LTN','24LTP','24LTQ','24LTR','24LUJ','24LUK',
      '24LUL','24LUM','24LUN','24LUP','24LUQ','24LUR','24LVL','24LVM',
      '24LVN','24LVP','24LVQ','24LVR','24LWM','24LWN','24LWP','24LWQ',
      '24LWR','24LXN','24LXP','24LXQ','24LXR','24LYP','24LYQ','24LYR',
      '24LZR','24MTA','24MTB','24MTC','24MTS','24MTT','24MTU','24MTV',
      '24MUA','24MUB','24MUC'
    ],

    'lsTilesBa': [
      '23LLD', '23LLE', '23LLF', '23LLG', '23LLH', '23LLJ', '23LMD', '23LME', '23LMF', '23LMG', '23LMH', '23LMJ',
      '23LND', '23LNE', '23LNF', '23LNG', '23LNH', '23LNJ', '23LPD', '23LPE', '23LPF', '23LPG', '23LPH', '23LPJ',
      '23LPK', '23LQD', '23LQE', '23LQF', '23LQG', '23LQH', '23LQJ', '23LQK', '23LRC', '23LRD', '23LRE', '23LRF',
      '23LRG', '23LRH', '23LRJ', '23LRK', '23LRL', '24KUE', '24KUF', '24KUG', '24KVE', '24KVF', '24KVG', '24LTK',
      '24LTL', '24LTM', '24LTN', '24LTP', '24LTQ', '24LTR', '24LUH', '24LUJ', '24LUK', '24LTH', '24LTJ', '24LUL',
      '24LUM', '24LUN', '24LUP', '24LUQ', '24LUR', '24LVH', '24LVJ', '24LVK', '24LVL', '24LVM', '24LVN', '24LVP',
      '24LVQ', '24LVR', '24LWH', '24LWJ', '24LWK', '24LWL', '24LWM', '24LWN', '24LWP', '24LWQ', '24LWR', '24LXM',
      '24LXN', '24LXP', '24LXQ'
    ],

    'imgRefCaat': 'COPERNICUS/S2_SR/20200809T131249_20200809T131246_T23MQP'
}

In [5]:
params = {
    "ccobert": 60,
    "start": None,
    "end": None,   
    "mes": None,
    "ano": 2020,
    "bandasAll": ['B2','B3', 'B4', 'B8', 'B11', 'B12'],        
    "listclassesMB": [1,2,3,4,5,9,10,12], 
    "idassetOut": 'users/Tarefa01_MAPBIOMAS/teste_alerta_caatinga/ver1/',
    "gradeS2": 'users/solkancengine17/shapes/grade_sentinel_brasil',
    "gradeS2Corr": 'projects/mapbiomas-arida/ALERTAS/auxiliar/shpGradeNordeC',
    "assetMapbiomas": "projects/mapbiomas-workspace/public/collection5/mapbiomas_collection50_integration_v1",
    "folderROIs": {'id':'projects/mapbiomas-arida/ALERTAS/ROIs/trainingF/'},
    "folderGroupROIs": 'projects/mapbiomas-arida/ALERTAS/ROIs/trainingV42/',
    "classeIds": [1,2,3,4,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33], 
    "classeIdsNew": [1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
    "limiarImg": 4,    
    "Feature" :['osavi','ndwi','ndvi','lai', 'soil',  'gcvi', 'npv', 'ndfia'], 
    "FeatureSVM": ['ndwi', 'lai', 'soil'],   
    "allBand": ['osavi','ndwi','ndvi','lai', 'soil',  'gcvi', 'npv', 'ndfia','class'],  
    'pathHome': "/home/superusuario/Dados/projAlertas/tabFeitas/",
    'path_TF': "/tabFeitas/",
    'numeroTask': 0,
    'numeroLimit': 85,
    'isCaatinga': True,
    'conta' : {
        '0': 'caatinga01',
        '10': 'caatinga02',
        '20': 'caatinga03',
        '24': 'caatinga04',
        '32': 'caatinga05',        
        '42': 'solkan1201',
        '54': 'diegoGmail',
        '27': 'rodrigo',
        '64': 'Rafael',
        '75': 'Nerivaldo',
        #'39': 'solkanCengine',
        # '45': 'soltangalano',
        # '45': 'ellen'        
    },
}

In [16]:
class ClassCalcIndicesSpectral(object):

    # default options
    bndInd = []  
    options = {        
        "bandas": ['B2','B3', 'B4', 'B8', 'B11', 'B12'],        
        # "Feature" :['ndwi', 'osavi', 'lai', 'soil', 'evi','gv','ndfia'], 
        "Feature" :['osavi','ndwi','ndvi','lai', 'soil',  'gcvi', 'npv', 'ndfia'], 
        'bandsFraction': ['gv','npv','soil','cloud', 'shade'] 
        # "pmtRF" : {'numberOfTrees': 50, 'variablesPerSplit': 3,  'seed': 2}
    }  

    dictClassifRef = {}

    geomet = None

    imgUltima = None

    maskMapbiomas = None

    classificadorTrained = None

    # cloudsMascara = True

    imgColClouds = None

    def __init__(self, idRef):

        self.imgRef_ = ee.Image(idRef)
        
        geomRef = self.imgRef_.geometry()

        self.imgRef_  = self.imgRef_.select(self.options['bandas'])              
    
        self.imgRef_  = self.strecht_Images(self.imgRef_, geomRef)

        # salvando em um diccionario as propiedades dde Classify da ref
        self.equalizeRef(geomRef)
        
    
    def updateParametros(self, uImg, mMapBiomas, geomet):
        
        self.imgUltima = uImg

        self.maskMapbiomas = mMapBiomas.set('system:footprint', geomet)

        self.geomet = geomet

    def maskS2clouds(self, imgP):

        iDimgP = imgP.id()        
        imgMask = ee.Image(self.imgColClouds.filter(ee.Filter.eq('system:index', iDimgP)).first())

        # creando Mascara para pixels uties         
        return  imgP.updateMask(imgMask.lt(15))  
    
    def NomalizeImg(self, image, bnd, geomet):
        
        imgTemp = image.select(bnd)        

        optRed = {
            'reducer': ee.Reducer.minMax(),
            'geometry': geomet,
            'scale': 10,
            'maxPixels': 1e13
        } 

        extremos =  ee.Dictionary(imgTemp.reduceRegion(**optRed))
            
        imgTemp = imgTemp.unitScale(extremos.getNumber(bnd + "_min"), extremos.getNumber(bnd + "_max"))\
                    .multiply(10000).uint16()

        imgTemp = imgTemp.rename(bnd)

        return imgTemp

    def strecht_Images(self, image, geomet):        

        matching = ee.Image().int16()  

        for band in self.options['bandas']:

            imgTemp = self.NomalizeImg(image, band, geomet)

            imgTemp = imgTemp.rename(band)        

            matching = matching.addBands(imgTemp)

        matching = ee.Image.cat(matching.select(self.options['bandas'])) 

        # matching = matching.set()       

        return matching

    def getFC(self, image_, bnd, geomet):
        
        #Histogram equalization start:      
            
        optRed = {
            'reducer': ee.Reducer.histogram(maxBuckets= 10000),
            'geometry': geomet,
            'scale': 10,
            'maxPixels': 1e13,
            'tileScale': 4
            }

        histo = image_.reduceRegion(**optRed)         

        valsList = ee.List(ee.Dictionary(histo.get(bnd)).get('bucketMeans'))    

        freqsList = ee.List(ee.Dictionary(histo.get(bnd)).get('histogram'))

        cdfArray = ee.Array(freqsList).accum(0)

        total = cdfArray.get([-1])

        normalizedCdf = cdfArray.divide(total)

        lists = valsList.zip(normalizedCdf.toList())

        expLists = lists.reduce(ee.Reducer.toCollection(['dn', 'probability']))

        return ee.FeatureCollection(expLists)
    
    def equalizeRef(self, geomR):

        for bnd in self.options['bandas']:           

            propertImgRef = self.getFC(self.imgRef_.select(bnd), bnd, geomR)        

            ClassfImgRef = ee.Classifier.smileRandomForest(50).setOutputMode('REGRESSION')\
                .train(features= propertImgRef, classProperty= 'dn', inputProperties= ['probability'])

            self.dictClassifRef[bnd] = ClassfImgRef
    
    def equalize(self, image_, bnd, geomet):        

        propertImg = self.getFC(image_.select(bnd), bnd, geomet)        

        Classfimg = ee.Classifier.smileRandomForest(50).setOutputMode('REGRESSION')\
            .train(features= propertImg, classProperty= 'probability', inputProperties= ['dn'])

        imgExport = image_.select(bnd).rename('dn')

        #print "ja rodou a classification"
        # classifier from DigitalNivel   ---> probability  depois # classifier from probability   ---> DigitalNivel
        
        return imgExport.classify(Classfimg, 'probability')\
            .classify(self.dictClassifRef[bnd], bnd)    

    def match_Images(self, image):

        geom = image.geometry()
        
        matching = ee.Image().int16()    
        
        for band in self.options['bandas']:

            imgTemp = self.equalize(image, band, geom)
            
            imgTemp = imgTemp.toInt()

            matching = matching.addBands(imgTemp)

        matching = ee.Image.cat(matching.select(self.options['bandas']))

        return matching
    

    #Indices
    def agregateBandsIndexNDVI(self, img):
    
        ndviImg = img.expression("float(b('B8') - b('B12')) / (b('B8') + b('B12'))")\
            .rename(['ndvi'])       

        return img.addBands(ndviImg)

    #Novo
    def agregateBandsIndexGNDVI(self, img):
    
        gndviImg = img.expression("float(b('B8') - b('B3')) / (b('B8') + b('B3'))")\
        .rename(['gndvi'])       

        return img.addBands(gndviImg)

    def agregateBandsIndexEVI(self, img):
            
        eviImg = img.expression(
            "float(2.4 * (b('B8') - b('B4')) / (1 + b('B8') + b('B4')))")\
                .rename(['evi'])     
        
        return img.addBands(eviImg)
    
    def agregateBandsIndexLAI(self, img):
    
        laiImg = img.expression(
            "(3.618 * float(b('evi') - 0.118))")\
                .rename(['lai'])     
    
        return img.addBands(laiImg)
    
    def agregateBandsIndexGCVI(self, img):
    
        gcviImgA = img.expression(
            "float(b('B8')) / (b('B3')) - 1").divide(10)\
                .rename(['gcvi'])        
        
        return img.addBands(gcviImgA)
    
    def agregateBandsIndexOSAVI(self,img):
    
        osaviImg = img.expression(
            "float(b('B8') - b('B4')) / (0.16 + b('B8') + b('B4'))")\
                .rename(['osavi'])        
        
        return img.addBands(osaviImg)
    
    def agregateBandsIndexSoil(self, img):
        
        soilImg = img.expression(
            "float(b('B8') - b('B3')) / (b('B8') + b('B3'))")\
                .rename(['soil'])       
        
        return img.addBands(soilImg)    

    def agregateBandsIndexBAI(self, img):
    
        baiImg = img.expression(
            "float(1) / ((0.1 - b('B4'))**2 + (0.06 - b('B8'))**2)")\
                .rename(['bai']) 
        
        return img.addBands(baiImg)
    
    #Novo
    def agregateBandsIndexIFV(self, img):
    
        ifvImg = img.expression("float( 2 * b('B3') - b('B4') - b('B2')) / (2 * b('B3') + b('B4') + b('B2'))")\
        .rename(['ifv'])       

        return img.addBands(ifvImg)
    
    #Novo
    def agregateBandsIndexRATIO(self, img):
    
        ratioImg = img.expression("float(b('B8') / b('B4'))")\
        .rename(['ratio'])      

        return img.addBands(ratioImg)
    
    #Novo
    def agregateBandsIndexRVI(self, img):
    
        rviImg = img.expression("float(b('B4') / b('B8'))")\
        .rename(['rvi'])       

        return img.addBands(rviImg)
    
    def agregateBandsTexturasGLCM(self, img):
        
        img = img.toInt()
                
        textura2 = img.select('B8').glcmTexture(3)        

        entropia = textura2.select('B8_ent').divide(10000).rename('ent')            
        variancia = textura2.select('B8_var').divide(10000).rename('var')

        img = img.addBands(entropia).addBands(variancia)
        
        return  img.select(['ent','var'])
    
    #Novos Indices Rafael
    
    def agregateBandsIndexNDWI(self, img):
    
        ndwiImg = img.expression("float(b('B3') - b('B8')) / (b('B3') + b('B8'))")\
        .rename(['ndwi'])       

        return img.addBands(ndwiImg)
    
    #Óxidos de Ferro

    def agregateBandsIndexFEO(self, img):
    
        feoImg = img.expression("float(b('B11')) / (b('B8'))")\
        .rename(['feo'])       

        return img.addBands(feoImg) 
        
    def agregateBandsgetFractions(self, img):

        # Define endmembers
        endmembers =  [
            [ 119.0,  475.0,  169.0, 6250.0, 2399.0,  675.0], #/*gv*/
            [1514.0, 1597.0, 1421.0, 3053.0, 7707.0, 1975.0], #/*npv*/
            [1799.0, 2479.0, 3158.0, 5437.0, 7707.0, 6646.0], #/*soil*/
            [4031.0, 8714.0, 7900.0, 8989.0, 7002.0, 6607.0], #/*cloud*/
            [   0.0,    0.0,    0.0,    0.0,    0.0,    0.0]  #/*Shade*/
        ]

        # Uminxing data
        fractions = ee.Image(img).select(self.options['bandas'])\
                        .unmix(endmembers= endmembers).float()
        
        fractions = fractions.select([0,1,2,3,4], self.options['bandsFraction'])        
        return img.addBands(fractions)


    def agregateBandsIndexNDFIA(self, img):

        #calculate NDFI
        ndfia = img.expression(
            "float(b('gv') - b('soil')) / float( b('gv') + 2 * b('npv') + b('soil'))")
        
        ndfia = ndfia.rename('ndfia')
        
        return img.select(['npv']).addBands(ndfia) # 'gv',

    def CalculateIndice(self, imageW):         
        
        geomet = imageW.geometry()
        # imagem em Int16 com valores inteiros ate 10000        
        imageF = self.agregateBandsgetFractions(imageW)
        imageF = self.agregateBandsIndexNDFIA(imageF)
        # imageT = self.agregateBandsTexturasGLCM(imageW)
        imageW = imageW.divide(10000)
        imageW = imageW.set('system:footprint', geomet)

        imageW = self.agregateBandsIndexEVI(imageW)         
        #imageW = self.agregateBandsIndexWater(imageW) 
        imageW = self.agregateBandsIndexNDVI(imageW)               
        imageW = self.agregateBandsIndexOSAVI(imageW)
        imageW = self.agregateBandsIndexGCVI(imageW)
        imageW = self.agregateBandsIndexSoil(imageW)        
        imageW = self.agregateBandsIndexLAI(imageW)

        #Novo
        imageW = self.agregateBandsIndexGNDVI(imageW)
        imageW = self.agregateBandsIndexIFV(imageW)
        imageW = self.agregateBandsIndexRATIO(imageW)
        imageW = self.agregateBandsIndexRVI(imageW)

        #Indices Rafael
        imageW = self.agregateBandsIndexFEO(imageW)
        imageW = self.agregateBandsIndexNDWI(imageW)
        
        return imageW.addBands(imageF)
        

In [17]:
def exportarClassification(imgTransf, nameAl, geomet):
    
    #noReg = 2
    
    # IdAsset = 'projects/mapbiomas-arida/ALERTAS/alertas-brutos/creados/' + nameAl    
    IdAsset = 'projects/mapbiomas-arida/ALERTAS/alertas-brutos/gerados/' + nameAl
    print(" en id Asset:")
    print("   <> {}".format(IdAsset))
    
    optExp = {
        'image': imgTransf, #.toInt16()
        'description': nameAl, 
        'assetId':IdAsset, 
        'pyramidingPolicy': {".default": "mode"},  
        'region': geomet.getInfo()['coordinates'],
        'scale': 10,
        'maxPixels': 1e13 
    }

    task = ee.batch.Export.image.toAsset(**optExp)    
    task.start()
    print("reporte do status")

    for keys, vals in dict(task.status()).items():
        print ( "  {} : {}".format(keys, vals))
    print ("salvando ... !")

In [18]:
tiles['imgRefCaat']

'COPERNICUS/S2_SR/20200809T131249_20200809T131246_T23MQP'

In [9]:
lsIndMin = []
lsIndMax = []

if params['isCaatinga']:
    lsTiles = tiles['lsTilesCaat']
else:
    lsTiles = tiles['lsTilesBa']

params['start'] = '2020-01-01'
params['end'] = '2020-12-31'

sufixo = ''
# Seco _dry
# chuvoso = _wet

gradeS2 = ee.FeatureCollection(params['gradeS2'])

datasetSent2 = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterDate(params['start'], params['end'])\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', params['ccobert']))\
    .select(params["bandasAll"])

datasetCloudS2 = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
    .filterDate(params['start'], params['end'])

print('Imagem de referencia \n ====> ' + tiles['imgRefCaat'])
operadorMosaic = ClassCalcIndicesSpectral(tiles['imgRefCaat'])
operadorMosaic.imgColClouds = datasetCloudS2

Imagem de referencia 
 ====> COPERNICUS/S2_SR/20200809T131249_20200809T131246_T23MQP


In [19]:
dictArqReg = {
        '9': [
                '24LYP', '24LYQ', '24LYR', '24LZR', '24MZS', '24MZT', '24MZU', '24MZV', 
                '25LBL', '25MBM', '25MBN', '25MBP', '25MBQ'
        ],
        '38': [
                '23LMC', '23LMG', '23LMH', '23LMJ', '23LND', '23LNE', '23LNF', '23LNG', 
                '23LNH', '23LNJ', '23LNK', '23LNL', '23LPH', '23LPJ', '23LPK', '23LPL', 
                '23MNM', '23MPM', '23MPN', '23MQN', '23MQP', '23MRS', '23MRT'
        ]
}

In [12]:
dictStd_IndBND = {
        'median': [
                    'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'evi2', 'ndvi', 
                    'ndwi', 'savi', 'pri', 'gcvi', 'hallcover', 'cai', 'gv', 'gvs', 
                    'npv', 'soil', 'cloud', 'shade', 'ndfi', 'sefi', 'wefi', 'fns',
                    "awei","iia"
        ],

        'stdDev':[
                    'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'evi2', 'ndvi', 
                    'ndwi', 'savi', 'gcvi', 'hallcover', 'cai', 'gv', 'gvs', 
                    'npv', 'soil', 'cloud', 'shade', 'ndfi', 'sefi', 'wefi', 'fns'
        ],
        
        'min': [
                    'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'evi2', 'ndvi', 
                    'gcvi', 'npv', 'soil'
        ],

        'max': [
                    'ndvi', 'ndwi', 'savi', 'gcvi', 'npv', 'soil'
        ],

        'amp': [
                    'nir', 'swir2', 'evi2', 'ndvi', 'ndwi', 'savi', 'pri', 'gcvi',
                    'npv', 'soil', 'ndfi'
        ],

        'dry': [
                    'blue', 'green', 'red', 'nir', 'swir1', 'evi2', 'ndvi', 'ndwi', 
                    'savi', 'pri', 'gcvi', 'cai', 'gv', 'npv', 'soil','ndfi', 'sefi', 
                    'fns'
        ],

        'wet': [
                    'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'evi2', 'ndvi', 
                    'ndwi', 'savi', 'pri', 'gcvi', 'gv', 'npv', 'soil', 'ndfi', 'wefi',
                    'fns'
        ],

}

In [25]:
for orb, listTiles in dictArqReg.items():

    for tile in listTiles[:1]:  

        geomet = gradeS2.filter(ee.Filter.eq('NAME', tile)).geometry() 
        newDataset = datasetSent2.filter(ee.Filter.eq('MGRS_TILE', tile)).filter(ee.Filter.eq('MGRS_TILE', tile))

        #print(newDataset.min())

        numImg = newDataset.size().getInfo()
        print("Para a orbita <<{}>> e tile << {} >> temos # {}  imagens".format(orb, tile, numImg))

        lsIndexImg = newDataset.reduceColumns(ee.Reducer.toList(), ['system:index']).get('list').getInfo()

        #for nameId in lsIndexImg:
        #    print(nameId) 

        #remoção de Nuvens
        newDataset = newDataset.map(lambda image: operadorMosaic.maskS2clouds(image))
        ##  matchiong histogram 
        newDataset = newDataset.map(lambda image: operadorMosaic.match_Images(image))

        ## Clac
        newDatasetInd = newDataset.map(lambda image: operadorMosaic.CalculateIndice(image))

        dataset = newDatasetInd.first().getInfo()
        lsBndEsp = newDatasetInd.first().bandNames().getInfo()
        #bands = newDatasetInd.first().select(features).getInfo()

        ## Reducers Meand
        imgAnalitic = ee.Image().float()
        reducer = 'median_'       
        for bnd in lsBndEsp:
          bandTemp = ee.Image(newDatasetInd.select(bnd).mean()).rename(reducer + bnd)
          bandTemp = bandTemp.add(1).multiply(10000).toUint16()
          imgAnalitic = imgAnalitic.addBands(bandTemp)

        ##Reducers Minimum
        reducer = 'min_'
        for bnd in lsIndMin:
          bandTemp = ee.Image(newDatasetInd.select(bnd).min()).rename(reducer + bnd)
          bandTemp = bandTemp.add(1).multiply(10000).toUint16()
          imgAnalitic = imgAnalitic.addBands(bandTemp)
        
        ##Reducers Maximum
        reducer = 'max_'
        for bnd in lsIndMax:
         bandTemp = ee.Image(newDatasetInd.select(bnd).max()).rename(reducer + bnd)
         bandTemp = bandTemp.add(1).multiply(10000).toUint16()
         imgAnalitic = imgAnalitic.addBands(bandTemp)
        
        #set properties
        imgAnalitic = imgAnalitic.clip(geomet)
        imgAnalitic = imgAnalitic.set('system:footprint', geomet)
        imgAnalitic = imgAnalitic.set('year', params['ano'])
        imgAnalitic = imgAnalitic.set('MGRS_TILE', tile)
        imgAnalitic = imgAnalitic.set('NUM_IMAGENS', numImg)

        # save imagens 
        nameAl = str(params['ano']) + '_' + tile
        # exportarClassification(imgAnalitic, nameAl, geomet)
        print(nameAl)

Para a orbita <<9>> e tile << 24LYP >> temos # 110  imagens
2020_24LYP
Para a orbita <<38>> e tile << 23LMC >> temos # 105  imagens
2020_23LMC
